In [ ]:
!pip install ../input/package/pretrainedmodels-0.7.4/ -f ./ --no-index
#!pip install dlib
import os
from os.path import join

#import dlib
import cv2
import torch
import torch.nn as nn
import argparse
import sys
import time
sys.path.append("/kaggle/input")
sys.path.append("/kaggle/input/faceforensics")
from tqdm import tqdm
from PIL import Image as pil_image
from torchvision import transforms
import pretrainedmodels


xception_default_data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
}


In [ ]:
import numpy as np
dirname = '/kaggle/input/deepfake-detection-challenge/test_videos'
max_confidence = 0.99
min_confidence = 0.01

In [ ]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb


def preprocess_image(image, cuda=True):
    """
    Preprocesses the image such that it can be fed into our network.
    During this process we envoke PIL to cast it into a PIL image.

    :param image: numpy image in opencv form (i.e., BGR and of shape
    :return: pytorch tensor of shape [1, 3, image_size, image_size], not
    necessarily casted to cuda
    """
    # Revert from BGR
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Preprocess using the preprocessing function used during training and
    # casting it to PIL image
    preprocess = xception_default_data_transforms['test']
    preprocessed_image = preprocess(pil_image.fromarray(image))
    # Add first dimension as the network expects a batch
    preprocessed_image = preprocessed_image.unsqueeze(0)
    if cuda:
        preprocessed_image = preprocessed_image.cuda()
    return preprocessed_image


def predict_with_model(image, model, post_function=nn.Softmax(dim=1),
                       cuda=True):
    """
    Predicts the label of an input image. Preprocesses the input image and
    casts it to cuda if required

    :param image: numpy image
    :param model: torch model with linear layer at the end
    :param post_function: e.g., softmax
    :param cuda: enables cuda, must be the same parameter as the model
    :return: prediction (1 = fake, 0 = real)
    """
    # Preprocess
    preprocessed_image = preprocess_image(image, cuda)

    # Model prediction
    output = model(preprocessed_image)
    output = post_function(output)

    return output

In [ ]:
def test_full_image_network_(video_path, model, face_detector,
                            start_frame=0, end_frame=None, cuda=True):
    # Read and write
    reader = cv2.VideoCapture(video_path)

    fps = reader.get(cv2.CAP_PROP_FPS)
    num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Frame numbers and length of output video
    frame_num = 0
    assert start_frame < num_frames - 1
    end_frame = end_frame if end_frame else num_frames
    frame_logits = []
    
    while reader.isOpened():
        frame_num += 1
        print('{}/{}'.format(frame_num, num_frames), end="\r")
        _ = reader.grab()
        if image is None:
            break
        if frame_num < start_frame or frame_num % 3 == 0:
            continue
        
        _, image = reader.retrieve()
        # Image size
        height, width = image.shape[:2]

        # 2. Detect with dlib
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 1)

        if len(faces):
            # For now only take biggest face
            face = faces[0]

            # --- Prediction ---------------------------------------------------
            # Face crop with dlib and bounding box scale enlargement
            x, y, size = get_boundingbox(face, width, height)
            cropped_face = image[y:y+size, x:x+size]

            # Actual prediction using our model
            logit = predict_with_model(cropped_face, model, cuda=cuda)[0, 1].detach().cpu().item()
            frame_logits.append(logit*2)
            # ------------------------------------------------------------------

        if frame_num >= end_frame:
            break
            
    num_detected_faces = len(frame_logits)
    video_logit = sum(frame_logits)
    if num_detected_faces > 0: 
        video_logit = video_logit/num_detected_faces
    return np.clip(video_logit, min_confidence, max_confidence)

In [ ]:
def get_boundingbox_v(face, width, height, mode='dlib', scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    if mode == 'dlib':
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        size_bb = int(max(x2 - x1, y2 - y1) * scale)
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
    elif mode == 'opencv':
        x1, y1, w, h = face
        size_bb = int(max(w, h) * scale)
        center_x = x1 + w//2
        center_y = y1 + h//2
    else:
        raise Exception("Invalid mode!!", mode)
        
    if minsize:
        if size_bb < minsize:
            size_bb = minsize

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb

def test_image_opencv(video_path, model, face_detector, mode='dlib',
                            start_frame=0, end_frame=None, cuda=True):
    # Read and write
    reader = cv2.VideoCapture(video_path)

    fps = reader.get(cv2.CAP_PROP_FPS)
    num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Frame numbers and length of output video
    frame_num = 0
    assert start_frame < num_frames - 1
    end_frame = end_frame if end_frame else num_frames
    frame_logits = []
    
    while reader.isOpened():
        frame_num += 1
        if frame_num < start_frame or frame_num % 3 != 1:
            continue
        print('{}/{}'.format(frame_num, num_frames), end="\r")
        
        _ = reader.grab()
        _, image = reader.retrieve()
        
        if image is None:
            break
        # Image size
        height, width = image.shape[:2]
        
        #start = time.time()
        # 2. Detect with dlib or opencv
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if mode == 'dlib':
            faces = face_detector(gray, 1)
        elif mode == 'opencv':
            faces = face_detector.detectMultiScale(gray, 1.8, 3, minSize=(50,50), maxSize=(500,500))
        else:
            raise Exception("Invalid mode!!", mode)
        #end = time.time()
        #print("Detect Face: ", end-start, " || Len Faces: ", len(faces))
        
        if len(faces):
            # For now only take biggest face
            face = faces[0]

            # --- Prediction ---------------------------------------------------
            # Face crop with dlib and bounding box scale enlargement
            x, y, size = get_boundingbox_v(face, width, height, mode='opencv')
            cropped_face = image[y:y+size, x:x+size]

            # Actual prediction using our model
            logit = predict_with_model(cropped_face, model, cuda=cuda)[0, 1].detach().cpu().item()
            frame_logits.append(logit*2)
            # ------------------------------------------------------------------
        
        if frame_num >= end_frame:
            break
            
    num_detected_faces = len(frame_logits)
    if (num_detected_faces < 10): 
        return 0.5
    video_logit = sum(frame_logits)
    if num_detected_faces > 0: 
        video_logit = video_logit/num_detected_faces
    return np.clip(video_logit, min_confidence, max_confidence)

In [ ]:
cuda = True
print("Open Cuda: ", cuda)

#face_detector = dlib.get_frontal_face_detector()

face_cascade = cv2.CascadeClassifier()
if not face_cascade.load("/kaggle/input/haarcascade/haarcascade_frontalface_alt.xml"):
    raise Exception("Cannot load face_cascade")

model = torch.load("/kaggle/input/faceforensics/all_c40.p", map_location="cpu")

if cuda: 
    model = model.cuda()

with open("submission.csv", "w") as f:
    f.write("filename,label\n")
    for i, filename in enumerate(os.listdir(dirname)):
        video_path = os.path.join(dirname, filename)
        # print('progress: {}, video path: {}'.format(i+1, video_path))
        start = time.time()
        logit = test_image_opencv(video_path, model, face_cascade, mode='opencv', cuda=cuda)
        end = time.time()
        # print()
        print(video_path.split('/')[-1], ": ", logit, " || Using Time: ", end-start)
        f.write("{},{}\n".format(filename, logit))

# Any results you write to the current directory are saved as output.